In [1]:
import re
import json
from parse_bitstream_v2 import BitstreamParser
import hashlib
from collections import defaultdict

In [31]:
import config_command

In [32]:

config_command.asf()

AttributeError: module 'config_command' has no attribute 'asf'

In [2]:
json_file_path = r"E:\workspace\parse_bitstream\parse_bitstream\压缩位流相关知识\MCF100T_v2.json"
# 读取JSON文件
with open(json_file_path, 'r') as file:
    json_file = json.load(file)

In [3]:
file_path = r"E:\workspace\parse_bitstream\parse_bitstream\压缩位流相关知识\compress\原版\origin.rbt"
bit_parser = BitstreamParser(file_path, "False")

In [4]:
data_frame = bit_parser.rbt_data_content

In [5]:
# 获取特征值
# 这里传入content为list， 每个元素的数据类型为 content_type
def get_feature(content:list, content_type = "int"):
    if content_type == "str":
        # 此时是二进制字符串作为一个元素
        combined_string = ''.join(content)
        # 使用 hashlib 计算特征值（SHA256）
        feature_hash = hashlib.sha256(combined_string.encode()).hexdigest()
        return feature_hash
    elif content_type == "int":
        pass

In [10]:
a = "00000000000000000001010000000000"
print(len(a))
print(a)

32
00000000000000000001010000000000


In [15]:
b = a[:-11] + '0' + a[-10:]
print(len(b))
print(b)

32
00000000000000000001000000000000


In [16]:
a = [1,2,3,9,10,11,12]
a.append

32
00000000000000000000010000000000
